In [26]:
%matplotlib inline

import pandas as pd
import numpy as np
import pylab as plt
import seaborn as sns

from sklearn.model_selection import train_test_split as tts
from sklearn.metrics import mean_squared_error as mse

from sklearn.ensemble import RandomForestRegressor as RFR  
from sklearn.tree import ExtraTreeRegressor as ETR
from sklearn.tree import DecisionTreeRegressor as DTR
from sklearn.ensemble import GradientBoostingRegressor as GBR
from xgboost import XGBRegressor as XGBR
from catboost import CatBoostRegressor as CTR
from lightgbm import LGBMRegressor as LGBMR

from xgboost import XGBRFRegressor as XGBRF
from sklearn.linear_model import LinearRegression as LinReg
from sklearn.neighbors import KNeighborsRegressor as KNR
from sklearn.linear_model import LogisticRegression as LogReg

import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)

In [27]:
salaries= pd.read_csv('../data/salaries_data.csv')

test = pd.read_csv('../data/testeo.csv')

In [28]:
#Borramos columnas que no hace falta
test.drop('salary', inplace=True, axis=1)
#test.drop('salary_currency', inplace=True, axis=1)

In [29]:
salaries.head()

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,2022,SE,FT,Data Engineer,140250,USD,140250,US,100,US,M
1,2022,SE,FT,Data Engineer,135000,USD,135000,US,100,US,M
2,2021,MI,FT,BI Data Analyst,100000,USD,100000,US,100,US,M
3,2021,MI,CT,ML Engineer,270000,USD,270000,US,100,US,L
4,2021,MI,FT,Data Engineer,22000,EUR,26005,RO,0,US,L


In [30]:
def diction(col):
    lista = list(set(list(salaries[col].unique()) + list(test[col].unique())))
    nums =[i for i in range(len(lista))]
    print({(lista[i]): (nums[i]) for i in range(len(lista))})
    return {(lista[i]): (nums[i]) for i in range(len(lista))}

dictio_experience_level = diction('experience_level')
dictio_employment_type = diction('employment_type')
dictio_job_title = diction('job_title')
dictio_employee_residence = diction('employee_residence')
dictio_company_location = diction('company_location')
dictio_company_size = diction('company_size')
dictio_salary_currency = diction('salary_currency')

{'EX': 0, 'EN': 1, 'SE': 2, 'MI': 3}
{'FT': 0, 'CT': 1, 'PT': 2, 'FL': 3}
{'Machine Learning Manager': 0, 'Machine Learning Developer': 1, 'Data Architect': 2, 'BI Data Analyst': 3, 'Analytics Engineer': 4, 'Data Analytics Manager': 5, 'Cloud Data Engineer': 6, 'Data Specialist': 7, 'Business Data Analyst': 8, 'Research Scientist': 9, 'Lead Data Analyst': 10, 'ETL Developer': 11, 'Big Data Architect': 12, 'Data Engineering Manager': 13, 'Applied Machine Learning Scientist': 14, 'NLP Engineer': 15, 'Data Analytics Engineer': 16, 'Big Data Engineer': 17, 'Director of Data Engineering': 18, 'Data Analytics Lead': 19, 'Data Engineer': 20, 'Principal Data Analyst': 21, 'Computer Vision Software Engineer': 22, 'Data Scientist': 23, 'Finance Data Analyst': 24, 'Director of Data Science': 25, 'Machine Learning Engineer': 26, 'Machine Learning Scientist': 27, 'Head of Machine Learning': 28, 'Product Data Analyst': 29, 'Applied Data Scientist': 30, 'Marketing Data Analyst': 31, 'Data Science Man

In [31]:
salaries_copy_1 = salaries.copy(deep=True)
salaries_copy_1.head()

salaries_copy_1.drop(['salary'], inplace=True,axis=1)
print(salaries_copy_1.head())

test_copy_1 = test.copy(deep=True)
print(test_copy_1.head())

   work_year experience_level employment_type        job_title  \
0       2022               SE              FT    Data Engineer   
1       2022               SE              FT    Data Engineer   
2       2021               MI              FT  BI Data Analyst   
3       2021               MI              CT      ML Engineer   
4       2021               MI              FT    Data Engineer   

  salary_currency  salary_in_usd employee_residence  remote_ratio  \
0             USD         140250                 US           100   
1             USD         135000                 US           100   
2             USD         100000                 US           100   
3             USD         270000                 US           100   
4             EUR          26005                 RO             0   

  company_location company_size  
0               US            M  
1               US            M  
2               US            M  
3               US            L  
4               US

In [32]:
test_copy_1.shape, salaries_copy_1.shape

((107, 9), (500, 10))

In [33]:
test_copy_1['y'] = -1
test_copy_1.head()

,work_year,experience_level,employment_type,job_title,salary_currency,employee_residence,remote_ratio,company_location,company_size,y
0,2020,SE,FT,Machine Learning Scientist,USD,JP,0,JP,S,-1
1,2020,MI,FT,Lead Data Analyst,USD,US,100,US,L,-1
2,2020,MI,FT,Data Analyst,USD,US,100,US,L,-1
3,2020,MI,FT,Machine Learning Engineer,CNY,CN,0,CN,M,-1
4,2020,MI,FT,Product Data Analyst,INR,IN,100,IN,L,-1


In [34]:
test_copy_1.shape, salaries_copy_1.shape

((107, 10), (500, 10))

In [35]:
todo = pd.concat([test_copy_1, salaries_copy_1])

In [36]:
todo.job_title = todo.job_title.apply(lambda x: dictio_job_title[x])
todo.employment_type = todo.employment_type.apply(lambda x: dictio_employment_type[x])
todo.employee_residence = todo.employee_residence.apply(lambda x: dictio_employee_residence[x])
todo.experience_level = todo.experience_level.apply(lambda x: dictio_experience_level[x])
todo.company_size = todo.company_size.apply(lambda x: dictio_company_size[x])
todo.company_location = todo.company_location.apply(lambda x: dictio_company_location[x])
todo.salary_currency = todo.salary_currency.apply(lambda x: dictio_salary_currency[x])
todo.head()

,work_year,experience_level,employment_type,job_title,salary_currency,employee_residence,remote_ratio,company_location,company_size,y,salary_in_usd
0,2020,2,0,27,9,47,0,40,2,-1.0,NaN
1,2020,3,0,10,9,46,100,39,1,-1.0,NaN
2,2020,3,0,49,9,46,100,39,1,-1.0,NaN
3,2020,3,0,26,6,7,0,7,0,-1.0,NaN
4,2020,3,0,29,0,5,100,5,1,-1.0,NaN


In [37]:
ok = todo.loc[todo['y'] == -1]
ok.head()

,work_year,experience_level,employment_type,job_title,salary_currency,employee_residence,remote_ratio,company_location,company_size,y,salary_in_usd
0,2020,2,0,27,9,47,0,40,2,-1.0,NaN
1,2020,3,0,10,9,46,100,39,1,-1.0,NaN
2,2020,3,0,49,9,46,100,39,1,-1.0,NaN
3,2020,3,0,26,6,7,0,7,0,-1.0,NaN
4,2020,3,0,29,0,5,100,5,1,-1.0,NaN


In [38]:
ok.shape

(107, 11)

In [39]:
ok_s = todo.loc[todo['y'] != -1]
ok_s.head()

,work_year,experience_level,employment_type,job_title,salary_currency,employee_residence,remote_ratio,company_location,company_size,y,salary_in_usd
0,2022,2,0,20,9,46,100,39,0,NaN,140250.0
1,2022,2,0,20,9,46,100,39,0,NaN,135000.0
2,2021,3,0,3,9,46,100,39,0,NaN,100000.0
3,2021,3,1,42,9,46,100,39,1,NaN,270000.0
4,2021,3,0,20,5,32,0,39,1,NaN,26005.0


In [40]:
ok_s.shape

(500, 11)

In [41]:
ok_s.drop('y', axis=1, inplace=True)
ok.drop('salary_in_usd', axis=1, inplace=True)
ok.drop('y', axis=1, inplace=True)

## Modelos

In [42]:
X = ok_s.drop('salary_in_usd', axis = 1)
y = ok_s.salary_in_usd

In [43]:
X_train, X_test, y_train, y_test = tts(X, y, train_size=0.99, test_size=0.01, random_state=42)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((495, 9), (5, 9), (495,), (5,))

In [44]:
#Inicializamos modelos
rfr=RFR()
etr=ETR()
gbr=GBR()
xgbr=XGBR()
ctr=CTR()
dtr=DTR()
lgbmr=LGBMR()
xgbrf = XGBRF()
linreg = LinReg()
knr = KNR()
logreg = LogReg()

#Entrenamientos
rfr.fit(X_train, y_train)
etr.fit(X_train, y_train)
gbr.fit(X_train, y_train)
xgbr.fit(X_train, y_train)
ctr.fit(X_train, y_train, verbose=0)
dtr.fit(X_train, y_train)
lgbmr.fit(X_train, y_train)
xgbrf.fit(X_train, y_train)
linreg.fit(X_train, y_train)
knr.fit(X_train, y_train)
logreg.fit(X_train, y_train)

#Predicciones
y_pred_rfr = rfr.predict(X_test)
y_pred_etr = etr.predict(X_test)
y_pred_gbr = gbr.predict(X_test)
y_pred_xgbr = xgbr.predict(X_test)
y_pred_ctr = ctr.predict(X_test)
y_pred_dtr = dtr.predict(X_test)
y_pred_lgbmr = lgbmr.predict(X_test)
y_pred_xgbrf = xgbrf.predict(X_test)
y_pred_linreg = linreg.predict(X_test)
y_pred_knr = knr.predict(X_test)
y_pred_logreg = logreg.predict(X_test)

#Calculo de error
print(mse(y_test, y_pred_rfr, squared=False), mse(y_test, y_pred_etr, squared=False), mse(y_test, y_pred_gbr, squared=False), 
mse(y_test, y_pred_xgbr, squared=False), mse(y_test, y_pred_ctr, squared=False), mse(y_test, y_pred_dtr, squared=False),
mse(y_test, y_pred_lgbmr, squared=False), mse(y_test, y_pred_xgbrf, squared=False), mse(y_test, y_pred_linreg, squared=False),
mse(y_test, y_pred_knr, squared=False), mse(y_test, y_pred_logreg, squared=False))

29436.485469502473 31107.05042561317 33480.51147098285 29559.56859985629 29130.896412837414 31107.05042561317 29772.334968091258 36992.741354246085 31186.790550923113 23681.11686758038 33381.519438156196


### Mejor Modelo: KNeighborsRegressor

In [45]:
knr.fit(X_train, y_train)

y_pred_knr = knr.predict(ok)
y_pred_knr[:10]

len(y_pred_knr)

107

## Exportar

In [46]:
res = pd.read_csv('../data/muestra.csv')
res.salary_in_usd = y_pred_knr
res.head()

,id,salary_in_usd
0,0,192459.2
1,1,146400.0
2,2,121800.0
3,3,72436.8
4,4,35203.8


In [47]:
res.to_csv('../data/resultado_knr_contodo_0.99.csv', index = False)